# Experiencia número 2: Luckas Diaz - Diego Pincheira

# Configuraciones Preliminares:
Se cargan las librerías requeridas. Para esta experiencia, se requiere el uso de la librería **natsorted**.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import color, io, exposure
import os
# pip install natsort # Se debe tener instalado natsorted para ejecutar el código
from natsort import natsorted

## Carga de archivos
Se suben los archivos **ojos_gal.rar**, **ojos_impostor.rar** y **ojos_test.rar** a Google Colab.

In [2]:
!wget https://github.com/DiegoPincheiraIb/EL5206_Assignments/raw/master/Experiencia_2/ojos_gal.rar
!wget https://github.com/DiegoPincheiraIb/EL5206_Assignments/raw/master/Experiencia_2/ojos_impostor.rar
!wget https://github.com/DiegoPincheiraIb/EL5206_Assignments/raw/master/Experiencia_2/ojos_test.rar

--2021-08-22 01:12:52--  https://github.com/DiegoPincheiraIb/EL5206_Assignments/raw/master/Experiencia_2/ojos_gal.rar
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DiegoPincheiraIb/EL5206_Assignments/master/Experiencia_2/ojos_gal.rar [following]
--2021-08-22 01:12:53--  https://raw.githubusercontent.com/DiegoPincheiraIb/EL5206_Assignments/master/Experiencia_2/ojos_gal.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 192354 (188K) [application/octet-stream]
Saving to: ‘ojos_gal.rar’

ojos_gal.rar        100%[===================>] 187.85K  --.-KB/s    in 0.03s   

2021-08-22 01:12:53 (5.97 MB

Se descomprimen los archivos.

In [3]:
!unrar x ojos_gal.rar
!unrar x ojos_impostor.rar
!unrar x ojos_test.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from ojos_gal.rar

Creating    ojos_gal                                                  OK
Extracting  ojos_gal/I10_gal.jpg                                           9%  OK 
Extracting  ojos_gal/I1_gal.jpg                                           19%  OK 
Extracting  ojos_gal/I2_gal.jpg                                           31%  OK 
Extracting  ojos_gal/I3_gal.jpg                                           42%  OK 
Extracting  ojos_gal/I4_gal.jpg                                           50%  OK 
Extracting  ojos_gal/I5_gal.jpg                                           60%  OK 
Extracting  ojos_gal/I6_gal.jpg                                           71%  OK 
Extracting  ojos_gal/I7_gal.jpg                                           79%  OK 
Extracting  ojos_gal/I8_gal.jpg                                           90%  OK 


Se verifica que los archivos se hayan cargado.

In [7]:
!ls

ojos_gal      ojos_impostor	 ojos_test	sample_data
ojos_gal.rar  ojos_impostor.rar  ojos_test.rar


Se crea un string con la ruta del directorio, y se ordenan los archivos de forma que el primer sujeto en la carpeta respectiva corresponda al sujeto listado como número 1 de su carpeta (esto debido a que, por defecto, no se da así.). Para esta experiencia, **es indispensable que los sujetos se encuentren ordenados.**

In [8]:
Str_Gal = 'ojos_gal/'
Str_Impostor = 'ojos_impostor/ojos_impostor/'
Str_Test = 'ojos_test/'
Listado_Gal = os.listdir(Str_Gal) # Se crea un puntero al directorio
Listado_Gal = natsorted(Listado_Gal) # Se ordenan los archivos con el número del sujeto de menor a mayor.
Listado_Impostor = os.listdir(Str_Impostor)
Listado_Impostor = natsorted(Listado_Impostor)
Listado_Test = os.listdir(Str_Test)
Listado_Test = natsorted(Listado_Test)

## Carga y preprocesamiento de imágenes

Se crean tres listas, **Listado_Gal_Gris**, **Listado_Impostor_Gris**, y **Listado_Test_Gris**, las cuales contienen la información de los tres subconjuntos a analizar en esta tarea: **Ojos_Gal**, **Ojos_Impostor** y **Ojos_test**. El procedimiento para cargar las imágenes dentro de cada lista es el siguiente:

1.   Se carga la imagen mediante **io.imread()**
2.   Se convierte a escala de grises mediante **color.rgb2gray()**.
3.   Se aplica *ecualización adaptativa de histograma* mediante **exposure.equalize_adapthist()**.
4.   Se agrega la matriz generada a la lista.





In [10]:
def Generador_Listado(listado_input:list, str_obj: str) -> list:
  output = []
  for imagen in listado_input:
    print(str_obj + imagen)
    output.append(exposure.equalize_adapthist(color.rgb2gray(io.imread(str_obj + imagen))))
  return output

In [11]:
print('Procesamiento del conjunto ''Ojos_Gal'':')
Listado_Gal_Gris = Generador_Listado(Listado_Gal, Str_Gal)

print('---------------------------------------------')

print('Procesamiento del conjunto ''Ojos_Impostor'':')
Listado_Impostor_Gris = Generador_Listado(Listado_Impostor, Str_Impostor)

print('---------------------------------------------')

print('Procesamiento del conjunto ''Ojos_Test'':')
Listado_Test_Gris = Generador_Listado(Listado_Test, Str_Test)


Procesamiento del conjunto Ojos_Gal:
ojos_gal/I1_gal.jpg
ojos_gal/I2_gal.jpg
ojos_gal/I3_gal.jpg
ojos_gal/I4_gal.jpg
ojos_gal/I5_gal.jpg
ojos_gal/I6_gal.jpg
ojos_gal/I7_gal.jpg
ojos_gal/I8_gal.jpg
ojos_gal/I9_gal.jpg
ojos_gal/I10_gal.jpg
---------------------------------------------
Procesamiento del conjunto Ojos_Impostor:
ojos_impostor/ojos_impostor/imp_0001.jpg
ojos_impostor/ojos_impostor/imp_0002.jpg
ojos_impostor/ojos_impostor/imp_0003.jpg
---------------------------------------------
Procesamiento del conjunto Ojos_Test:
ojos_test/I1_test.jpg
ojos_test/I2_test.jpg
ojos_test/I3_test.jpg
ojos_test/I4_test.jpg
ojos_test/I5_test.jpg
ojos_test/I6_test.jpg
ojos_test/I7_test.jpg
ojos_test/I8_test.jpg
ojos_test/I9_test.jpg
ojos_test/I10_test.jpg


# Desarrollo P1
Algoritmo LBP

La función **LBP()** permite aplicar el algoritmo *Local binary patterns* a una imagen dada. La imagen resultante posee 2 pixeles menos de ancho y alto que la imagen original para no aplicar *padding* a la imagen original.

In [12]:
def LBP(ImagenGris_Matriz):
  Matriz_Final_Copia = np.copy(ImagenGris_Matriz)
  Mascara_Valores = np.array([[128., 64., 32.],
                  [1., 0., 16.],
                  [2., 4., 8.]])
  for row in range(1,len(ImagenGris_Matriz)-1):
    for column in range(1,len(ImagenGris_Matriz[0])-1):
      mask = np.zeros((3,3))
      mask[1,1] = ImagenGris_Matriz[row,column]
      for fila in range(-1,2):
        for columna in range(-1,2):
          if fila != 0 and columna !=0:                      
            if ImagenGris_Matriz[row+fila,column+columna] > mask[1,1]:
              mask[fila, columna] = 1
      Numero_Entero = np.sum(mask * Mascara_Valores)
      Matriz_Final_Copia[row, column] = Numero_Entero

  Matriz_Final_Copia_Copia = np.delete(Matriz_Final_Copia, -1, 0) # Elimina la última fila
  Matriz_Final_Copia = np.delete(Matriz_Final_Copia_Copia, -1, 1) # Elimina la última columna
  Matriz_Final_Copia_Copia = np.delete(Matriz_Final_Copia, 0, 0) # Elimina la primera fila
  Matriz_Final_Copia = np.delete(Matriz_Final_Copia_Copia, 0, 1) # Elimina la primera columna
  return Matriz_Final_Copia

Se crea la función **Imagen_Plot()**, que genera un gráfico en escala de grises a partir de la imagen objetivo.

In [13]:
def Imagen_Plot(Matriz_Obj):
  plt.imshow(Matriz_Obj, cmap='gray')
  plt.show()

### Aplicación de LBP a los datos:
Se aplica LBP a los datos, y se guardan en listas.

In [14]:
Gal_LBP = []
Impostor_LBP = []
Test_LBP = []

for pic in Listado_Gal_Gris:
  Gal_LBP.append(LBP(pic))

for pic in Listado_Impostor_Gris:
  Impostor_LBP.append(LBP(pic))

for pic in Listado_Test_Gris:
  Test_LBP.append(LBP(pic))

**¿Qué características tienen las imágenes procesadas con LBP?**

Una imagen procesada con LBP otorga una **mejor visualizacion** para los detalles mas finos y las texturas de la imagen, permitiendo una **mejor extracción de características**. Por otra parte, como resultado del procedimiento anterior, los pixeles tendran una intensidad entre [0,255], lo que permite elaborar un histograma de 256 bins a partir de la información.

# Desarrollo P2

## Extraer_Caract():
Dado una imagen de 500x600, devuelve un **vector con histogramas resultantes de dividir la imagen en 30 regiones**. Esta función permite *normalizar los histogramas* que se calculan si se le entrega un booleano "True" como argumento adicional.

In [15]:
def Extraer_Caract(Imagen, Normalizar=False):
  Hist_Arr = np.zeros((0,))
  # se recorre la matriz de la imagen para dividir en 30 zonas
  for row in range(0, Imagen.shape[0], 100): 
    for col in range(0, Imagen.shape[1], 100):
      # se calcula el histograma de la zona actual
      hist, bin_edges = np.histogram(Imagen[row:row+100, col:col+100], bins=59, density=Normalizar)
      Hist_Arr = np.concatenate((Hist_Arr, hist))
  return Hist_Arr

# Desarrollo P3

## Distancia():
Permite calcular la distancia punto a punto entre dos vectores dados con un mismo largo.

In [16]:
def distancia(vec1, vec2):
  assert len(vec1) == len(vec2)
  n = len(vec1)
  dist = 0
  for i in range(0, n):
    dist = dist + abs(vec1[i] - vec2[i])
  return dist

# Desarrollo P4

## Base de datos:

Utilizando la variable Gal_LBP_List que contiene la transformacion LBP de las imágenes del archivo ojos_gal.zip, se construye una base de datos con los vectores de características de las imágenes contenidas en el archivo.

In [17]:
DB_Gal = []

for matriz in Gal_LBP: # Se recorre la lista de las imágenes de Ojos_gal
  # Se calcula el vector de características de la imagen
  Gal_caract = Extraer_Caract(matriz, True)
  # Se almacena el vector de características calculado en una lista
  DB_Gal.append(Gal_caract)

# Desarrollo P5

### Reconocimiento():
Esta función identifica a sujetos cuyas matrices pertenecientes a los ojos de sujetos se encuentran contenidos en una lista, teniendo como referencia una determinada base de datos. Notese que **se asume que todos los sujetos a identificar se encuentran en la base de datos**, por lo que esta función se limita a **relacionar los sujetos del conjunto entrante con los contenidos en la base de datos**.

In [18]:
def Reconocimiento_P5(DB, lista):
  Lista_minimos = [] # Se guardarán los valores mínimos.
  Sujeto_Base_Datos = 1
  for matriz in lista:
    Lista_Comparacion = []
    test_caract = Extraer_Caract(matriz, True) # Extraer mi vector de características
    for caract in DB:      
      Distancia_Elementos = distancia(caract, test_caract) # Calculo la distancia entre los 2 vectores
      Lista_Comparacion.append(Distancia_Elementos) # Se guarda la distancia en un arreglo
    if Lista_Comparacion.index(min(Lista_Comparacion)) + 1 == Sujeto_Base_Datos:
      Lista_minimos.append(min(Lista_Comparacion)) # Se guarda la mínima distancia entre el vector de caract. de Test y el de la base de datos
    print('El sujeto ' + str(Sujeto_Base_Datos) +
        ' ha sido identificado como el sujeto ' + str(Lista_Comparacion.index(min(Lista_Comparacion)) + 1) +
        ' en nuestra base de datos. Distancia mínima: ' + str(min(Lista_Comparacion)))
    Sujeto_Base_Datos += 1
  print('Sujetos encontrados en la base de datos: ' + str(len(Lista_minimos)) + '/' + str(len(lista)))

Prueba de reconocimiento entre las imagenes del los archivos ojos_gal.zip y ojos_test.zip.

In [19]:
P5 = Reconocimiento_P5(DB_Gal, Test_LBP)

El sujeto 1 ha sido identificado como el sujeto 1 en nuestra base de datos. Distancia mínima: 9.899357142857141
El sujeto 2 ha sido identificado como el sujeto 2 en nuestra base de datos. Distancia mínima: 3.2959928571428603
El sujeto 3 ha sido identificado como el sujeto 3 en nuestra base de datos. Distancia mínima: 5.535042857142864
El sujeto 4 ha sido identificado como el sujeto 4 en nuestra base de datos. Distancia mínima: 7.567592857142867
El sujeto 5 ha sido identificado como el sujeto 5 en nuestra base de datos. Distancia mínima: 8.513700000000004
El sujeto 6 ha sido identificado como el sujeto 6 en nuestra base de datos. Distancia mínima: 10.341014285714298
El sujeto 7 ha sido identificado como el sujeto 7 en nuestra base de datos. Distancia mínima: 8.648978571428579
El sujeto 8 ha sido identificado como el sujeto 8 en nuestra base de datos. Distancia mínima: 10.150107142857154
El sujeto 9 ha sido identificado como el sujeto 9 en nuestra base de datos. Distancia mínima: 9.37046


Con lo anterior se determina que se realizan 10 reconocimientos correctos. Nótese que la distancia mínima **no es superior a d = 10.36**.

# Desarrollo P6
Se tiene ahora una lista (*Impostor_LBP*) con 3 individuos: 1 contenido en la base de datos (denominado *falso impostor*), y 2 individuos **que no se encuentran registrados en la base de datos**. Al aplicar la función creada anteriormente (*Reconocimiento_P5()*), se asociará cada sujeto con uno presente en la base de datos, a pesar que no esté contenido ahí, como se puede apreciar a continuación.

In [28]:
P6_Prueba = Reconocimiento_P5(DB_Gal, Impostor_LBP)

El sujeto 1 ha sido identificado como el sujeto 9 en nuestra base de datos. Distancia mínima: 13.906721428571451
El sujeto 2 ha sido identificado como el sujeto 2 en nuestra base de datos. Distancia mínima: 6.088378571428577
El sujeto 3 ha sido identificado como el sujeto 3 en nuestra base de datos. Distancia mínima: 15.473592857142863
Sujetos encontrados en la base de datos: 2/3


Se puede apreciar que, en efecto, se asoció cada uno de los sujetos de *Impostor_LBP* a sujetos dentro de la base de datos; sin embargo, las distancias para *dos de ellos* son *mayores* a las distancias típicas en comparación a los sujetos de *test_LBP*. Considerando esto, es posible **fijar un umbral para el valor de la distancia mínima** que permita discriminar si los sujetos se encuentran o no contenidos en la base de datos.

De esta manera, se modifica la función *Reconocimiento_P5()* añadiéndole un parámetro adicional **"umbral"**, que permite establecer como *impostores* aquellos sujetos cuya distancia mínima dentro de la base de datos sea **mayor** a este. Así, se tiene la función **Reconocimiento_P6()**:

In [21]:
def Reconocimiento_P6(DB, lista, umbral):
  Lista_minimos = []
  Sujeto_Base_Datos = 1
  for matriz in lista:
    Lista_Comparacion = []
    test_caract = Extraer_Caract(matriz, True)
    for caract in DB:      
      Distancia_Elementos = distancia(caract, test_caract)
      Lista_Comparacion.append(Distancia_Elementos)
    if min(Lista_Comparacion) < umbral:
      Lista_minimos.append(min(Lista_Comparacion))
      print('El sujeto ' + str(Sujeto_Base_Datos) +
          ' ha sido identificado como el sujeto ' + str(Lista_Comparacion.index(min(Lista_Comparacion)) + 1) +
          ' en nuestra base de datos. Distancia mínima: ' + str(min(Lista_Comparacion)))
    else:
      print('No se ha encontrado el sujeto ' + str(Sujeto_Base_Datos) +   ' en la base de datos. Es un Impostor!!!')
    Sujeto_Base_Datos += 1
  print('Sujetos encontrados en la base de datos: ' + str(len(Lista_minimos)) + '/' + str(len(lista)))

Por último, se fija **umbral = 10.36**, ya que se observó que, en la base de datos, los valores de las distancias para *Test_LBP* no superaban ese umbral.

Se tiene a continuación el reconocimiento a los sujetos de la lista *Impostor_LBP*:

In [25]:
P6 = Reconocimiento_P6(DB_Gal, Impostor_LBP, 10.36)

No se ha encontrado el sujeto 1 en la base de datos. Es un Impostor!!!
El sujeto 2 ha sido identificado como el sujeto 2 en nuestra base de datos. Distancia mínima: 6.088378571428577
No se ha encontrado el sujeto 3 en la base de datos. Es un Impostor!!!
Sujetos encontrados en la base de datos: 1/3


Se puede observar que, a partir de **la distancia mínima entre los vectores de características**, en conjunto con un **umbral determinado**, ha sido posible discriminar a aquellos impostores que no se encuentran en la base de datos. A su vez, se observa que el sujeto dos en la lista *Impostor_LBP* **sí** se encuentra dentro de la base de datos (esto de acuerdo a que **su distancia mínima entre un elemento dentro de la base de datos se encuentra bajo el umbral establecido**), por lo que se identifica a este como el **falso impostor**.

Por último, se corrobora la función Reconocimiento_P6() con la lista *Test_LBP*, con el objetivo de verificar su correcto funcionamiento:

In [26]:
Reconocimiento_P6(DB_Gal, Test_LBP, 10.36)

El sujeto 1 ha sido identificado como el sujeto 1 en nuestra base de datos. Distancia mínima: 9.899357142857141
El sujeto 2 ha sido identificado como el sujeto 2 en nuestra base de datos. Distancia mínima: 3.2959928571428603
El sujeto 3 ha sido identificado como el sujeto 3 en nuestra base de datos. Distancia mínima: 5.535042857142864
El sujeto 4 ha sido identificado como el sujeto 4 en nuestra base de datos. Distancia mínima: 7.567592857142867
El sujeto 5 ha sido identificado como el sujeto 5 en nuestra base de datos. Distancia mínima: 8.513700000000004
El sujeto 6 ha sido identificado como el sujeto 6 en nuestra base de datos. Distancia mínima: 10.341014285714298
El sujeto 7 ha sido identificado como el sujeto 7 en nuestra base de datos. Distancia mínima: 8.648978571428579
El sujeto 8 ha sido identificado como el sujeto 8 en nuestra base de datos. Distancia mínima: 10.150107142857154
El sujeto 9 ha sido identificado como el sujeto 9 en nuestra base de datos. Distancia mínima: 9.37046

Nótese que, con un umbral más bajo, se comienza a dejar algunos sujetos fuera de la base de datos.

In [27]:
Reconocimiento_P6(DB_Gal, Test_LBP, 8)

No se ha encontrado el sujeto 1 en la base de datos. Es un Impostor!!!
El sujeto 2 ha sido identificado como el sujeto 2 en nuestra base de datos. Distancia mínima: 3.2959928571428603
El sujeto 3 ha sido identificado como el sujeto 3 en nuestra base de datos. Distancia mínima: 5.535042857142864
El sujeto 4 ha sido identificado como el sujeto 4 en nuestra base de datos. Distancia mínima: 7.567592857142867
No se ha encontrado el sujeto 5 en la base de datos. Es un Impostor!!!
No se ha encontrado el sujeto 6 en la base de datos. Es un Impostor!!!
No se ha encontrado el sujeto 7 en la base de datos. Es un Impostor!!!
No se ha encontrado el sujeto 8 en la base de datos. Es un Impostor!!!
No se ha encontrado el sujeto 9 en la base de datos. Es un Impostor!!!
El sujeto 10 ha sido identificado como el sujeto 10 en nuestra base de datos. Distancia mínima: 5.486578571428575
Sujetos encontrados en la base de datos: 4/10
